In [1]:
#convert python txt <p1> <p2> file to json file format
import json
f = open("mapping_db_wk_clean.txt",'r',encoding='utf-8').read()
json_dict = {"values" : []}
for line in f.split("\n"):
    values = line.split(" ")
    if len(values) == 2 : db,wk = values
    json_dict["values"].append({"db" : db, "wk": wk})
with open("mapping_db_wk.json", "w") as outfile:
    json.dump(json_dict, outfile)

In [2]:
import json
#compare sameAs prop with clean db prop
with open('mapping_db_wk.json', 'r') as openfile:
    mapping_dict = json.load(openfile)

#load mapping into a dict to get constant acces time
db_sameAs_dict = {}
for i in mapping_dict["values"]:
    db_sameAs_dict[i["db"]] = i["wk"]

with open("../data/property_support.json") as json_file:
        property_dictionnary = json.load(json_file)
            


In [3]:
#filter prop count dictionary with sameAs dictionary
filtered_prop = {}
for i in db_sameAs_dict.keys():
    val = property_dictionnary.get(i)
    if val: filtered_prop[i] = val

#sort the dict
prop = dict(sorted(filtered_prop.items(), key=lambda item: item[1], reverse=True))
#keep only prop with support > 1000
prop = dict(filter(lambda x,: True if x[1] > 1000 else False, prop.items()))



In [4]:
db_sameAs_dict['<http://dbpedia.org/ontology/deathDate>']

'<http://www.wikidata.org/prop/P570>'

In [5]:
for i in prop.keys():
        print(f"{db_sameAs_dict[i]} sameAs {i} : {prop[i]}")


<http://www.wikidata.org/prop/P11143> sameAs <http://dbpedia.org/ontology/wikiPageID> : 1406259
<http://www.wikidata.org/prop/P2451> sameAs <http://dbpedia.org/ontology/birthDate> : 471625
<http://www.wikidata.org/prop/P570> sameAs <http://dbpedia.org/ontology/deathDate> : 170365
<http://www.wikidata.org/prop/P2907> sameAs <http://dbpedia.org/ontology/utcOffset> : 163232
<http://www.wikidata.org/prop/P6499> sameAs <http://dbpedia.org/ontology/populationTotal> : 154981
<http://www.wikidata.org/prop/P8497> sameAs <http://dbpedia.org/ontology/Person/height> : 111006
<http://www.wikidata.org/prop/P8483> sameAs <http://dbpedia.org/ontology/postalCode> : 106248
<http://www.wikidata.org/prop/P5286> sameAs <http://dbpedia.org/ontology/runtime> : 97818
<http://www.wikidata.org/prop/P1477> sameAs <http://dbpedia.org/ontology/originalName> : 70857
<http://www.wikidata.org/prop/P10708> sameAs <http://dbpedia.org/ontology/areaCode> : 66970
<http://www.wikidata.org/prop/P10673> sameAs <http://dbpedi

### Test values 

In [6]:
#import 
import time
from SPARQLWrapper import SPARQLWrapper, JSON
import re

In [7]:
#const
DBPEDIA = "http://dbpedia.org/sparql"
WIKIDATA = "https://query.wikidata.org/sparql"

In [8]:
#isbn - wdt:P957

In [9]:
def sparql_query(query : str, endpoint : str ) -> dict:
    sparql = SPARQLWrapper(
        endpoint
    )
    sparql.setReturnFormat(JSON)
    sparql.setQuery(query)

    try:
        ret = sparql.queryAndConvert()
        return ret
    except Exception as e:
        print(e)
        raise Exception("Error in query")

In [10]:
def query_value_sample(prop : str) -> str:
    query = """
                select ?e ?p ?v where {
                    values ?p {"""+prop+"""}.
                    ?e ?p ?v
                } limit 100
            """
    query = re.sub(r"\n|'","",query)
    return query


In [11]:
#find prop to query
db_prop = "<http://dbpedia.org/ontology/releaseDate>"
wk_prop = "<http://www.wikidata.org/prop/statement/P10673>"
result_db = sparql_query(query_value_sample(db_prop),DBPEDIA)
result_wk = sparql_query(query_value_sample(wk_prop),WIKIDATA)

In [12]:
date_i = []
for i,j in zip(result_db["results"]["bindings"],result_wk["results"]["bindings"]):
    #print(i)
    #print(i['v']['value'],j['v']['value'])
    date_i.append(i['v']['value'])

In [35]:
properties_pair_file ="../data/properties_pair.txt"
db_wk_pairs = []
with open(properties_pair_file, "r", encoding="utf-8") as f:
    for line in f.readlines():
        print(line.split(" "))
        if len(splited := line.split(" ")) == 2:
            print(splited)
            db_wk_pairs.append((splited[0], splited[1]))


['<http://dbpedia.org/ontology/releaseDate>', '<http://www.wikidata.org/prop/P10673>']
['<http://dbpedia.org/ontology/releaseDate>', '<http://www.wikidata.org/prop/P10673>']


In [ ]:
import typing
from functools import reduce 
import ijson
import json
import os

def property_list_cutting(l: list ,slice_size: int ) -> list[list[str]]:
    """From a list, return slice 2D with n sublist of size "slice_size"

    Args:
        l (_type_): huge 1D list
        slice_size (_type_): 2D list composed of n "slice_size" list

    Returns:
        list : slice_size
    """
    index = 0
    sliced_list = []
    if len(l) < slice_size: return l
    while index<len(l):
        sliced_list.append(l[index:index+slice_size])
        index+=slice_size
    return sliced_list

def query_generate_VALUES(slice: list) -> str:
    """From a list return a string composed of all element of the list joined

    Args:
        slice (list): 1D list

    Returns:
        str : return a string of the type " slice[0] slice[1] slice[2] ... "
    """
    func = lambda acc,x : acc+" "+str(x)
    acc = ""
    joined_string = reduce(func, slice, acc)
    return joined_string

#TODO
def read_sameAs_file(file: str, var_names : list[str] = ["DB","WK"]) -> tuple[set[str],set[str]]:
    """Generate a list of sameAs from Dbpedia and Wikidata
    Args:
        file (str): sameAs json file name
    """
    f_read = open(file, 'r', encoding="UTF-8")

    first,second = var_names#refactor with var name header from json file

    db_entity_list = set()
    wk_entity_list = set()

    for record in ijson.items(f_read, "results.bindings.item"):
        db_entity_list.add(f'<{record[first]["value"]}>')
        wk_entity_list.add(f'<{record[second]["value"]}>')

    f_read.close()
    return db_entity_list,wk_entity_list


def read_ttl_file(file: str) -> tuple[list[str],list[str],list[str]]:
    """Generate a list entity property value from a ttl file
    Args:
        file (str): sameAs json file name
    """
    f_read = open(file, 'r', encoding="UTF-8").read()

    entity = []
    prop = []
    value = []

    for line in f_read.split("\n"):
        _line = line.split("\t")
        if len(_line) == 3:
            e,p,v = _line
            entity.append(e)
            prop.append(p)
            value.append(v)
        
    return entity,prop,value


def read_result_file(file : str, output_file : str, prop_var_name : str) -> None:
    """Read a result file to fill a dictionary 

    Args:
        file (str): result file in json format
    """
    
    f_read = open(file, 'r', encoding="UTF-8")
    prop = prop_var_name #name of the sparql var
    properties_count_file = output_file #dict file name
    #if file exist open json file and load 
    if os.path.exists(properties_count_file):
        with open(properties_count_file) as json_file:
            property_dictionnary = json.load(json_file)
            
    #else start with empty dict
    else:
        property_dictionnary = {}

    for record in ijson.items(f_read, "results.bindings.item"):
        item = (f'<{record[prop]["value"]}>')
        prop_count = property_dictionnary.get(item)
        if prop_count: property_dictionnary[item] += 1
        else: property_dictionnary[item] = 1

    f_read.close()
    #dump new dict
    with open(properties_count_file, "w") as fp:
        json.dump(property_dictionnary,fp) 

def sparql_call(sparql_query: str, result_file : str, database_name : str = "dbpedia") -> int :
    """Call a jar file to execute a sparql query on a database for a specified query 

    Args:
        sparql_query (str): sparql query we want to execute on a database
    """
    jar_path = "~/../soulard/QueryHDT/SparqlToJSON.jar"
    database_path = {
        "dbpedia" : "~/../soulard/Graphs_HDT/DBpedia/DBpedia_en.hdt",
        "wikidata" : "~/../soulard/Graphs_HDT/Wikidata/Wikidata_final.hdt" 
    }
    dataset_path = database_path[database_name]
    #hdt_command = "nohup java -Xmx120G -Xms120G -jar "+jar_path+" "+dataset_path+" \""+sparql_query+"\" > "+result_file+" &"
    hdt_command = "java -jar "+jar_path+" "+dataset_path+" \""+sparql_query+"\" > "+result_file
    return os.system(hdt_command)

def clean_nohup_file(result_file : str) -> None:
    """Commands to remove unwanted string at the start of nohup resulting files

    Args:
        result_file (str): _description_

    Returns:
        _type_: _description_
    """
    code = 0
    new_res_file = result_file.split(".")[0]+"_clean.json"
    code += os.system("cp "+result_file+" "+new_res_file)
    code += os.system("awk 'NR > 3 { print }' < "+new_res_file+" > "+result_file) #remove first 3 lines of the file
    return code

def concat_result_files(result_file : str, output_file : str, var_names : list) -> None:
    """Concat the json file to the output file

    Args:
        result_file (str): json file 
        output_file (str): ttl format file, append at the end of file
    """
    
    e,p,v = var_names
    f_read = open(result_file, 'r', encoding="UTF-8")
    f_write = open(output_file, 'a', encoding="UTF-8")
    for record in ijson.items(f_read, "results.bindings.item"):
        entity = (f'<{record[e]["value"]}>')
        prop = (f'<{record[p]["value"]}>')
        value = (f'{record[v]["value"]}')
        f_write.write(entity+"\t"+prop+"\t"+value+"\n")
        
    f_read.close()
    f_write.close()





In [ ]:
def find_entity_for_specific_prop(sameAs_file : str, prop : str, database_name : str) -> None:
    
    result_file = "../data/result_entity_that_support_p.json"
    name_of_prop = prop.split("/")[-1][0:-1]
    output_file = "../data/prop_"+name_of_prop+"_support_db.ttl"
    slice_size = 2000 #number of values in the VALUES keyword in sparql
    db_entity_list, wk_entity_list = list(map(list,read_sameAs_file(sameAs_file))) #convert the sets back to maps
    if database_name == "dbpedia":
        sliced = property_list_cutting(db_entity_list,slice_size) #cut the big list into slice in a 2D list
    if database_name == "wikidata":
        sliced = property_list_cutting(wk_entity_list,slice_size) #cut the big list into slice in a 2D list
    query_values_strings = [] #list of string properties used for VALUES in the sparql query
    for i in sliced:
        query_values_strings.append(query_generate_VALUES(i)) #join all prop in each bucket in a single string
    
    #call with sparql to get all properties used for each entities
    for entity in query_values_strings:
        sparql_query = """select distinct ?e ?p ?v  where {
            values ?e { """+entity+""" }.
            bind("""+prop+""" as ?p)
            ?e ?p ?v.
            }  
            """
        sparql_query = re.sub(r"\n|'","",sparql_query)
        sparql_call(sparql_query, result_file, database_name)
        concat_result_files(result_file, output_file, var_names=["e","p","v"])

def count_property_support(sameAs_file : str, database_name : str) -> None:
    """
        Count the support for every property supported by entity that have a sameAs link
    """
    result_file = "../data/result_prop.json"
    dict_output_file = "../data/property_support.json"
    slice_size = 1000 #number of values in the VALUES keyword in sparql

    db_entity_list, _ = list(map(list,read_sameAs_file(sameAs_file))) #convert the sets back to maps
    
    sliced = property_list_cutting(db_entity_list,slice_size) #cut the big list into slice in a 2D list
    query_values_strings = [] #list of string properties used for VALUES in the sparql query
    for i in sliced:
        query_values_strings.append(query_generate_VALUES(i)) #join all prop in each bucket in a single string
    
    #call with sparql to get all properties used for each entities
    for entity in query_values_strings:
        sparql_query = """select distinct ?a ?b  where {
            values ?a { """+entity+""" }.
            ?a ?b ?_.
            }  
            """
        sparql_query = re.sub(r"\n|'","",sparql_query)
        
        sparql_call(sparql_query, result_file, database_name)

        read_result_file(result_file, dict_output_file, prop_var_name="b")

def find_entity_for_specific_prop_hdt_version(sameAs_file : str, query_service : str, prop : str, database_name : str) -> None:
    
    name_of_prop = prop.split("/")[-1][0:-1]
    output_file = "../data/prop_"+name_of_prop+"_support_db.json"
    database_path = {
    "dbpedia" : "~/../soulard/Graphs_HDT/DBpedia/DBpedia_en.hdt",
    "wikidata" : "~/../soulard/Graphs_HDT/Wikidata/Wikidata_final.hdt" 
    }
    db_entity_list, wk_entity_list = list(map(list,read_sameAs_file(sameAs_file))) #convert the sets back to maps
    if database_name == "dbpedia":
        entities = query_generate_VALUES(db_entity_list) #cut the big list into slice in a 2D list
    if database_name == "wikidata":
        entities = query_generate_VALUES(wk_entity_list) #cut the big list into slice in a 2D list


    sparql_query = """select distinct ?e ?p ?v  where {
        values ?e { """+entities+""" }.
        bind("""+prop+""" as ?p)
        ?e ?p ?v.
        }  
        """        
    sparql_query = re.sub(r"\n|'","",sparql_query)
    query_file = name_of_prop+"_query"
    with open(query_file,"w",encoding="utf-8") as f:
        f.write(sparql_query)

    #call with sparql to get all entities for a specific property
    hdt_query_command = "java -Xmx50G -Xms50G -jar "+query_service+" "+database_path+" "+query_file+" "+output_file
    os.system(hdt_query_command)
    print("##############################")
    print("#############DONE#############")
    print("#############DONE#############")
    print("#############DONE#############")
    print("##############################")

def get_sameAs_for_pop_hdt_version(ttl_file : str, query_service : str, prop : str, database_name : str) -> None:
    
    name_of_prop = prop.split("/")[-1][0:-1]
    output_file = "../data/prop_"+name_of_prop+"_support_db.json"
    database_path = {
    "dbpedia" : "~/../soulard/Graphs_HDT/DBpedia/DBpedia_en.hdt",
    "wikidata" : "~/../soulard/Graphs_HDT/Wikidata/Wikidata_final.hdt" 
    }
    entity,_,_ = list(map(list,read_ttl_file(ttl_file))) #convert the sets back to maps
    entities = query_generate_VALUES(entity) #cut the big list into slice in a 2D list
   
    sparql_query = """
        PREFIX owl: <http://www.w3.org/2002/07/owl#> 
        select distinct ?e ?v  where {
        values ?e { """+entities+""" }. 
        ?e owl:sameAs ?v.
        FILTER(strstarts(str(?v), 'http://www.wikidata.'))
        }  
        """ 
    #sparql_query = re.sub(r"\n|'","",sparql_query)
    query_file = name_of_prop+"_query"
    with open(query_file,"w",encoding="utf-8") as f:
        f.write(sparql_query)

    #call with sparql to get all entities for a specific property
    hdt_query_command = "java -Xmx50G -Xms50G -jar "+query_service+" "+database_path[database_name]+" "+query_file+" "+output_file
    os.system(hdt_query_command)
    print("##############################")
    print("#############DONE#############")
    print("#############DONE#############")
    print("#############DONE#############")
    print("##############################")

def find_entity_for_specific_prop_hdt_version(sameAs_file : str, query_service : str, prop : str, database_name : str) -> None:
    
    name_of_prop = prop.split("/")[-1][0:-1]
    output_file = "../data/prop_"+name_of_prop+"_support_db.json"
    database_path = {
    "dbpedia" : "~/../soulard/Graphs_HDT/DBpedia/DBpedia_en.hdt",
    "wikidata" : "~/../soulard/Graphs_HDT/Wikidata/Wikidata_final.hdt" 
    }
    db_entity_list, wk_entity_list = list(map(list,read_sameAs_file(sameAs_file))) #convert the sets back to maps
    if database_name == "dbpedia":
        entities = query_generate_VALUES(db_entity_list) #cut the big list into slice in a 2D list
    if database_name == "wikidata":
        entities = query_generate_VALUES(wk_entity_list) #cut the big list into slice in a 2D list


    sparql_query = """select distinct ?e ?p ?v  where {
        values ?e { """+entities+""" }.
        bind("""+prop+""" as ?p)
        ?e ?p ?v.
        }  
        """        
    sparql_query = re.sub(r"\n|'","",sparql_query)
    query_file = name_of_prop+"_query"
    with open(query_file,"w",encoding="utf-8") as f:
        f.write(sparql_query)

    #call with sparql to get all entities for a specific property
    hdt_query_command = "java -Xmx50G -Xms50G -jar "+query_service+" "+database_path+" "+query_file+" "+output_file
    os.system(hdt_query_command)
    print("##############################")
    print("#############DONE#############")
    print("#############DONE#############")
    print("#############DONE#############")
    print("##############################")
